# Medium

## Unique Paths

* https://leetcode.com/problems/unique-paths/description/
***
* Time Complexity:
    - naive: O(2$^{m * n}$)
        * for every cell, you have 2 options: you either go down or you go to the right
        * there are m * n cells and 2 options for each cell
    - topdown memo: O(m x n)
        * unlike the naive, you aren't repeating any work and return immediately so you only visit each cell once
    - bottom up: O(m x n)
        * you have 2 loops going from 1...m and 1...n
* Space Complexity:
    - naive: O(m + n)
        * you will have at most O(m + n) functions in the stack b/c you can go all the way down to the m-th row and go all the way to the right to the n-th column
    - topdown memo: O(m x n)
        * requires space for the 2D dp table
    - bottom up: O(m x n)
        * requires space for the 2D dp table
***
* naive:
    - basically dfs/backtracking
    - you either go down or you go right at each cell
    - if you reach the bottom-right where the finish line is, then increment the numWays
    - else, the number of ways to reach that cell is equal to the number of ways to reach the cell above and to the right of it
        * numWays[r][c] = numWays[r + 1][c] + numWays[r][c + 1]
* topdown memo:
    - during the dfs/backtracking, we can actually reach the same cells we've already traversed on so we're just repeating work
    - thus we can use a dp table to keep track of which cells we already have numWays on
    - if we reach a cell that we've seen, we just return the numWays without traversing it again
* bottom up:
    - the concept is the same as topdown memo but we realize a couple of things:
        1. we know that all the cells in the top rown, at row 0, are going to have 1 as their numWays
            * reason being, you cannot reach them unless you keep going right
        2. the same is true for all the cells in the first column, at col 0
            * you cannot reach those cells unless you keep going down
    - we can use that to our advantage and ask ourselves
        * we know that the numWays[r][c] = numWays[r + 1][c] + numWays[r][c + 1]
        * but since we start from the beginning and work our way to the bottom-right then this must be reversed
        * so our __recurrence relation__ must be:
            - __dp[r][c] = dp[r - 1][c] + dp[r][c - 1]__
            - since we've already calculated the cells in the first row and first col, we can use those tabulated values already to calculate newer cells

In [ ]:
/**
 * @param {number} m
 * @param {number} n
 * @return {number}
 */

 // naive dfs/backtracking
var uniquePaths = function(m, n) {
    let numWays = 0;

    const traverse = (r, c) => {
        if (r >= m || c >= n) {
            return;
        }
        if ((r === m - 1) && (c === n - 1)) {
            numWays++;
        }

        // go down
        traverse(r + 1, c);

        // or go right
        traverse(r, c + 1);
    }

    traverse(0, 0);
    return numWays;
};

// naive dfs/backtracking
var uniquePaths = function(m, n) {
    const traverse = (r, c, numWays) => {
        if (r < 0 || c < 0) {
            return 0;
        }
        if (r === 0 && c === 0) {
            numWays++;
            return numWays;
        }

        return traverse(r - 1, c, numWays) + traverse(r, c - 1, numWays);
    }

    return traverse(m - 1, n - 1, 0);
};

// topdown memo
var uniquePaths = function(m, n) {
    const dp = Array.from({length: m}, () => Array.from({length: n}));

    const traverse = (r, c, numWays) => {
        if (r < 0 || c < 0) {
            return 0;
        }
        if (r === 0 && c === 0) {
            numWays++;
            return numWays;
        }
        if (dp[r][c] !== undefined) {
            return dp[r][c];
        }

        dp[r][c] = traverse(r - 1, c, numWays) + traverse(r, c - 1, numWays);
        return dp[r][c];
    }

    // console.log({dp})
    return traverse(m - 1, n - 1, 0);
};

// bottom-up dp
var uniquePaths = function(m, n) {
    const dp = Array.from({length: m}, () => Array.from({length: n}));

    for (let r = 0; r < m; r++) {
        dp[r][0] = 1;
    }

    for (let c = 0; c < n; c++) {
        dp[0][c] = 1;
    }

    for (let r = 1; r < m; r++) {
        for (let c = 1; c < n; c++) {
            dp[r][c] = dp[r - 1][c] + dp[r][c - 1];
        }
    }

    return dp[m - 1][n - 1];
}


## Longest Common Subsequence

* https://leetcode.com/problems/longest-common-subsequence/description/
***
* Time Complexity:
    - naive: O(2$^{n}$)
        * if text1[i] !== text2[j], then we have 2 choices to traverse: [i + 1, j] or [i, j + 1]
            - essentially, we disregard text1[i] and instead look at subsequence from text1[i + 1...n]
            - or disregard text2[j] and look at subsequence from text2[j + 1...n]
    - topdown memo: O(m x n), where m = length of text1 and n = length of text2
        * by using a dp table, we immediately solve any overlapping subproblems
        * thus we only need to solve for all subproblems of [i,j]
            - and since i = m and j = n, we have to solve m x n total problems
    - bottom up: O(m x n)
        * similar to the topdown memo
        * we fill out a table and solve m x n total problems to get the answer
* Space Complexity:
    - naive: O(max(m, n))
        * uses recursion so we need space for the function stack
        * when text1[i] !== text2[j] we have to traverse on [i + 1, j] and [i, j + 1]
            - i is bounded by the length of text1 and j is bounded by the length of text2 since we return immediately when i and j are equal or greater than m and n respectively
            - therefore, the max height of our recursion will be the max length between text1 and text2
    - topdown memo: O(m x n)
        * requires space for the dp table which is O(m x n)
    - bottom up: O(m x n)
        * requires space for the dp table which is O(m x n)
***
* I figured out the bottom up dp algorithm first before the naive/top down so i'll start with that
* bottom up dp:
    - visualize filling out a table where the rows represent the subsequences of text1 and the columns are subsequences of text2
    - when we fill out the first row and the first column respectively, what are we actually doing?
        * for the first row, we are checking if text1[0] is a subsequence of text2[0 ... n]
        * for the first col, we are checking if text2[0] is a subsequence of text1[0...n]
        * if at any point the first char of either of these texts are a subsequence of a section of the other text, then the rest of this text also contains that subsequence:
            - e.g. text1 = 'bac', text2 = 'a'
            - our first row would look like: [0, 1, 1]
                * 'b' !== 'a'
                * 'a' === 'a'
                * 'c' !== 'a' BUT we know that the previous subsequence DOES contain at least 1 char so we include it even though the chars don't match
    - once we fill out the first row and first col, we can then look at the __recurrence relation__:
        * __dp[i][j] =__ 
            - __1 + dp[i - 1][j - 1], if text1[i] === text2[j]__
            - __Math.max(dp[i - 1][j], dp[i, j - 1]), if text1[i] !== text2[j]__
        * essentially, if the current strings match at i and j, then look at the previous subsequences of both and add 1 to them
            - thus we look at dp[i - 1][j - 1]
        * if they don't match, then we look at previous subsequences of either text
            - dp[i - 1][j] = look at previous subsequences of text1
            - dp[i][j - 1] = look at previous subsequences of text2
* naive algorithm:
    - essentially use the same recurrence relation but done recursively
* dp algorithm:
    - notice that we might be solving for the same [i,j] subsequences so we use a dp table to keep track of those values
    - if we have already solved for it, then just return it

In [1]:
/**
 * @param {string} text1
 * @param {string} text2
 * @return {number}
 */

 // naive algorithm
 // Time: O(2^n)
 // Space: O(max(m, n))
var longestCommonSubsequence = function(text1, text2) {
    if (text1 === text2) return text1.length;

    const traverse = (i, j) => {
        if (i >= text1.length || j >= text2.length) return 0;

        if (text1[i] === text2[j]) {
            return 1 + traverse(i + 1, j + 1);
        }
        else {
            return Math.max(
                traverse(i + 1, j),
                traverse(i, j + 1)
            );
        }
    }

    return traverse(0, 0);
};


// topdown memo
// Time: O(m x n)
// Space: O(m x n)
var longestCommonSubsequence = function(text1, text2) {
    if (text1 === text2) return text1.length;
    const m = text1.length;
    const n = text2.length;
    const dp = Array.from({length: m}, () => []);

    const traverse = (i, j) => {
        if (i >= m || j >= n) return 0;
        if (dp[i][j] !== undefined) {
            return dp[i][j];
        }

        if (text1[i] === text2[j]) {
            dp[i][j] = 1 + traverse(i + 1, j + 1);
        }
        else {
            dp[i][j] = Math.max(
                traverse(i + 1, j),
                traverse(i, j + 1)
            );
        }

        return dp[i][j];
    }

    return traverse(0, 0);
};

// bottom up
// Time: O(m x n)
// Space: O(m x n)
var longestCommonSubsequence = function(text1, text2) {
    if (text1 === text2) return text1.length;

    // create the dp table
    const m = text1.length;
    const n = text2.length;
    const dp = Array.from({length: m}, () => []);

    // fill out the first row and first col
    dp[0][0] = text1[0] === text2[0] ? 1 : 0;

    // row
    for (let i = 1; i < m; i++) {
        dp[i][0] = text1[i] === text2[0] ? 1 : dp[i - 1][0];
    }

    // col
    for (let j = 1; j < n; j++) {
        dp[0][j] = text1[0] === text2[j] ? 1 : dp[0][j - 1];
    }

    // looping through the entire table
    for (let i = 1; i < m; i++) {
        for (let j = 1; j < n; j++) {
            if (text1[i] === text2[j]) {
                dp[i][j] = 1 + dp[i - 1][j - 1];
            }
            else {
                dp[i][j] = Math.max(dp[i - 1][j], dp[i][j - 1]);
            }
        }
    }

    return dp[m - 1][n - 1];
}

## Best Time to Buy and Sell Stock with Cooldown

* https://leetcode.com/problems/best-time-to-buy-and-sell-stock-with-cooldown/description/
***
* Time Complexity: O(n)
    - basically we just traverse from [0...n]
    - our dp table helps us to maintain O(n) complexity b/c of the key
        * normally it would be around O(2$^{n}$) b/c for each index from 0 to n, we can make 2 decisions:
            - if we haven't bought, we can buy or cooldown
            - if we have bought, we can sell or cooldown
            - if we have already sold, we must cooldown but this case is taken care of by skipping over the next index and going to index + 2
* Space Complexity: O(n)
    - needs space for the dp table
***
* __FOR THESE STOCK TYPE QUESTIONS WE MUST BE AWARE OF THREE STATES: BUY, SELL, HOLD__
* if we draw out a decision tree, each node in the tree represents an index and we have several options
    - i = 0: we can either buy or hold
        * we can't sell b/c we haven't bought anything
        * and we can ALWAYS HOLD (just do nothing)
    - i = 1:
        * if we bought at i = 0, we can either sell or cooldown
        * if we hold at i = 0, we can either buy or cooldown
    - i = 2:
        * if we hold at i = 1, we can either buy or cooldown
        * if we bought at i = 1, we can either sell or cooldown
        * if we sold at i = 1, WE MUST COOLDOWN which means this index would've been skipped and we would've arrived at i = 3 if this was the case
* thus our __recurrence relation is so:__
    * __dp[i][canBuy] = Math.max(dp[i + 1][!canBuy], dp[i + 1][canBuy]), if canBuy = true__
        - basically if we are buying, then we have to look at the max ofif we sell or hold on the next stock
    * __dp[i][canBuy] = Math.max(dp[i + 2][!canBuy], dp[i + 1][canBuy]), if canBuy = false__
        - basically if we are selling, we HAVE to take a cooldown and look at the max profit starting at i + 2 and if we decided to cooldown and not sell at i

In [2]:
/**
 * @param {number[]} prices
 * @return {number}
 */

 // Time Complexity: O(n)
 // Space Complexity: O(n)
var maxProfit = function(prices) {
    if (prices.length === 1) return 0;

    // key = 'index-boolean', val = max profit
    // e.g. 0-true, 10
    const dp = new Map();

    const traverse = (i, canBuy) => {
        if (i >= prices.length) return 0;
        
        const key = `${i}${canBuy}`
        if (dp.has(key)) {
            return dp.get(key);
        }

        // for our stock we can either buy, sell, or hold (cooldown)
        // this basically represents branches in our decision tree
        // however, we know that the problem has 1 constraint:
        // you MUST cooldown after you sell for at least 1 day (or 1 index)
        // so in order to accommodate for this, we must keep track of this state somehow

        // regardless of if we've bought or sold before
        // we can ALWAYS choose the option to cooldown or hold
        let cooldown = traverse(i + 1, canBuy);
        
        // if we are allowed to buy
        if (canBuy) {
            // we subtract prices[i] b/c we are LOSING profit by buying the stock
            // we traverse to the next index and we set the boolean to false
            // b/c we've already bought and do NOT want to buy again
            let buy = traverse(i + 1, false) - prices[i];
            dp.set(key, Math.max(buy, cooldown));
        }
        else {
            // since we are selling the stock, the next state MUST be to cooldown
            // so that's why we move 2 indices away
            // and we add prices[i] b/c we MADE profit (maybe) by selling the stock
            // we also set buying to true here b/c we've already sold
            // we're allowed to do it after we've cooldowned
            let sell = traverse(i + 2, true) + prices[i];
            dp.set(key, Math.max(sell, cooldown));
        }

        return dp.get(key);
    }

    return traverse(0, true);
};